<h1 align="center"> Model Optimization (Random Forest) — Objective & Plan </h1>

## Objective
Improve forecast accuracy (lower RMSE/MAE/MAPE) for weekly sales by tuning a Random Forest model and standardizing a reproducible workflow.

## Scope
- Targets: `N02BE`, `M01AB`
- Features: safe feature set (lags + calendar)
- Validation: time-aware CV (`TimeSeriesSplit`)
- Comparison: before vs. after tuning (hold-out test)

## Metrics
- Primary: RMSE
- Secondary: MAE, MAPE

## Plan
1. Load processed data and reproduce the same train/test split.
2. Prepare **two feature set variants**:
   - Full-safe (all lag + calendar)
   - Reduced-safe (drop weak calendar + far/low-utility lags)
3. Set up `TimeSeriesSplit` with consistent folds.
4. Run hyperparameter search for Random Forest (Randomized → focused Grid).
5. Retrain best configuration on full train; evaluate on test.
6. (Optional) Evaluate Reduced-safe set similarly.
7. Save artifacts (best params, metrics) and summarize improvements.

## Acceptance
- Same time split as earlier notebooks.
- Clear “before vs. after” metrics table for both targets.
- Artifacts saved under `../results/` and `../models/`.


## 1) Data & Feature Set Preparation

We reproduce the same 80/20 chronological split and build two feature sets:

- **Full-safe:** all lag features + calendar (`weekofyear`, `month`, `quarter`, `year`).
- **Reduced-safe:** union of top features from permutation-importance (per target) + minimal calendar.  
  Rationale: keep only features with proven predictive contribution on the test set and drop weak calendar flags.

Sanity checks:
- No current-week base columns
- No rolling features
- Same split dates as previous notebooks


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- Paths
DATA_PATH = Path("../data/processed/pharma_sales_features_v2_clean.csv")
IMP_DIR = Path("../results/importance")

# --- Load dataset
df = pd.read_csv(DATA_PATH, parse_dates=["datum"], index_col="datum")

# --- Chronological split (80/20) - same as before
split_index = int(len(df) * 0.8)
train = df.iloc[:split_index].copy()
test = df.iloc[split_index:].copy()

print(f"Date range: {df.index.min()} → {df.index.max()}")
print(
    f"Train: {train.index.min()} → {train.index.max()}  |  Test: {test.index.min()} → {test.index.max()}"
)

# --- Targets and safe features
targets = ["N02BE", "M01AB"]
base_cols = ["M01AB", "M01AE", "N02BA", "N02BE", "N05B", "N05C", "R03", "R06"]

# Minimal calendar: drop weak flags (is_year_start/end)
calendar_keep = ["weekofyear", "month", "quarter", "year"]

# Candidate features = everything except targets
feature_cols = [c for c in df.columns if c not in targets]

# Full-safe = all lags + minimal calendar
full_safe = [c for c in feature_cols if (("_lag" in c) or (c in calendar_keep))]

# Sanity checks for leakage
assert not any(
    c in base_cols for c in full_safe
), "Leakage: base columns present in features!"
assert not any("roll" in c for c in full_safe), "Leakage: rolling features present!"

X_train_full = train[full_safe]
X_test_full = test[full_safe]

print("\n[Full-safe] Feature count:", len(full_safe))
print("Example features:", full_safe[:10])

# --- Reduced-safe: use permutation importance (if available) to select top features per target
# Strategy:
# - Load perm_N02BE.csv and perm_M01AB.csv from ../results/importance
# - Take top_k from each, union them, then add minimal calendar
top_k = 20  # you can adjust this based on desired sparsity

perm_n02be_path = IMP_DIR / "perm_N02BE.csv"
perm_m01ab_path = IMP_DIR / "perm_M01AB.csv"

reduced_set = set()

if perm_n02be_path.exists():
    perm_n02be = pd.read_csv(perm_n02be_path)
    top_n02be = (
        perm_n02be.sort_values("MeanDrop", ascending=False)
        .head(top_k)["Feature"]
        .tolist()
    )
    reduced_set.update(top_n02be)

if perm_m01ab_path.exists():
    perm_m01ab = pd.read_csv(perm_m01ab_path)
    top_m01ab = (
        perm_m01ab.sort_values("MeanDrop", ascending=False)
        .head(top_k)["Feature"]
        .tolist()
    )
    reduced_set.update(top_m01ab)

# Keep only features that are in full-safe (no leakage) and add minimal calendar
reduced_safe = sorted([c for c in reduced_set if c in full_safe] + calendar_keep)

# Fallback: if importance files are missing, fall back to a simple rule-based reduced set
if len(reduced_safe) <= len(calendar_keep):
    # Rule: keep lag1–lag4 for all ATC codes + minimal calendar
    lag_prefixes = base_cols  # each ATC code
    keep_rules = []
    for code in lag_prefixes:
        for k in [1, 2, 3, 4]:
            keep_rules.append(f"{code}_lag{k}")
    reduced_safe = sorted([c for c in keep_rules if c in df.columns] + calendar_keep)

# Final sanity checks
assert not any(c in base_cols for c in reduced_safe), "Leakage in reduced-safe!"
assert not any(
    "roll" in c for c in reduced_safe
), "Rolling feature leaked into reduced-safe!"

X_train_reduced = train[reduced_safe]
X_test_reduced = test[reduced_safe]

print("\n[Reduced-safe] Feature count:", len(reduced_safe))
print("Example features:", reduced_safe[:10])

# Shapes
print("\nShapes:")
print("X_train_full:", X_train_full.shape, " | X_test_full:", X_test_full.shape)
print(
    "X_train_reduced:",
    X_train_reduced.shape,
    " | X_test_reduced:",
    X_test_reduced.shape,
)

# Quick null checks
print("\nNull checks (should be zero):")
print(
    "Full-safe train/test:",
    X_train_full.isna().sum().sum(),
    X_test_full.isna().sum().sum(),
)
print(
    "Reduced-safe train/test:",
    X_train_reduced.isna().sum().sum(),
    X_test_reduced.isna().sum().sum(),
)

Date range: 2014-06-15 00:00:00 → 2019-10-13 00:00:00
Train: 2014-06-15 00:00:00 → 2018-09-16 00:00:00  |  Test: 2018-09-23 00:00:00 → 2019-10-13 00:00:00

[Full-safe] Feature count: 52
Example features: ['M01AB_lag1', 'M01AB_lag2', 'M01AB_lag3', 'M01AB_lag4', 'M01AB_lag8', 'M01AB_lag12', 'M01AE_lag1', 'M01AE_lag2', 'M01AE_lag3', 'M01AE_lag4']

[Reduced-safe] Feature count: 35
Example features: ['M01AB_lag2', 'M01AE_lag8', 'N02BA_lag1', 'N02BA_lag12', 'N02BA_lag4', 'N02BA_lag8', 'N02BE_lag1', 'N02BE_lag12', 'N02BE_lag2', 'N02BE_lag8']

Shapes:
X_train_full: (223, 52)  | X_test_full: (56, 52)
X_train_reduced: (223, 35)  | X_test_reduced: (56, 35)

Null checks (should be zero):
Full-safe train/test: 0 0
Reduced-safe train/test: 0 0


## 2) TimeSeriesSplit Setup

We apply `TimeSeriesSplit` to ensure chronological validation.  
Unlike random CV, this method preserves temporal order — each fold uses earlier data to predict later data.

Key points:
- `n_splits = 5` (approx. 1-year per fold)
- Expanding window strategy (no look-ahead)
- Evaluation metric: RMSE (root mean squared error)

This structure will be reused for hyperparameter tuning and model comparison.


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

# --- Setup time series CV
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)


# --- Helper function for RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


# --- Dry run to confirm split sizes
fold_sizes = []
for i, (train_idx, val_idx) in enumerate(tscv.split(X_train_full)):
    fold_sizes.append((len(train_idx), len(val_idx)))
    print(f"Fold {i+1}: train={len(train_idx)}, val={len(val_idx)}")

print("\nFold size summary:")
print(fold_sizes)

Fold 1: train=38, val=37
Fold 2: train=75, val=37
Fold 3: train=112, val=37
Fold 4: train=149, val=37
Fold 5: train=186, val=37

Fold size summary:
[(38, 37), (75, 37), (112, 37), (149, 37), (186, 37)]


## 3) Randomized Hyperparameter Search (Random Forest, full-safe)

We tune RF to reduce RMSE using time-aware CV:
- CV: TimeSeriesSplit(n_splits=5)
- Scoring: negative RMSE (custom scorer)
- Space: n_estimators, max_depth, min_samples_split/leaf, max_features


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import pandas as pd
from pathlib import Path

OPT_DIR = Path("../results/optimization")
OPT_DIR.mkdir(parents=True, exist_ok=True)


# --- custom negative RMSE scorer
def neg_rmse(y_true, y_pred):
    return -np.sqrt(mean_squared_error(y_true, y_pred))


rmse_scorer = make_scorer(neg_rmse, greater_is_better=False)

# --- search space
param_distributions = {
    "n_estimators": [200, 300, 400, 600, 800],
    "max_depth": [None, 8, 12, 16, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", 0.5, 0.7],
    "bootstrap": [True],
    "random_state": [42],
}


def run_random_search_fullsafe(target, X_train, y_train, cv, n_iter=30):
    rf = RandomForestRegressor(n_jobs=-1, random_state=42)
    rs = RandomizedSearchCV(
        rf,
        param_distributions=param_distributions,
        n_iter=n_iter,
        scoring=rmse_scorer,
        cv=cv,
        n_jobs=-1,
        refit=True,
        random_state=42,
        verbose=0,
    )
    rs.fit(X_train, y_train)
    # results
    cvres = pd.DataFrame(rs.cv_results_)
    out_csv = OPT_DIR / f"rf_randomsearch_fullsafe_{target}.csv"
    cvres.to_csv(out_csv, index=False)
    best_rmse = -rs.best_score_
    print(f"[{target}] Best CV RMSE: {best_rmse:.3f}")
    print(f"[{target}] Best params: {rs.best_params_}")
    return rs, best_rmse


# --- run for both targets on FULL-SAFE features
rs_n02be, best_rmse_n02be = run_random_search_fullsafe(
    "N02BE", X_train_full, train["N02BE"], tscv
)
rs_m01ab, best_rmse_m01ab = run_random_search_fullsafe(
    "M01AB", X_train_full, train["M01AB"], tscv
)

[N02BE] Best CV RMSE: -56.268
[N02BE] Best params: {'random_state': 42, 'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 16, 'bootstrap': True}
[M01AB] Best CV RMSE: -9.621
[M01AB] Best params: {'random_state': 42, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 0.7, 'max_depth': 8, 'bootstrap': True}


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pathlib import Path

RESULTS_DIR = Path("../results/comparison")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)


def mape(y_true, y_pred):
    eps = 1e-8
    denom = np.clip(np.abs(y_true), eps, None)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100


# Best estimators from RandomizedSearchCV
best_rf_n02be = rs_n02be.best_estimator_
best_rf_m01ab = rs_m01ab.best_estimator_

# Retrain on full train (zaten refit=True ile eğitilmiş durumda; yine de tutarlılık için fit edebiliriz)
best_rf_n02be.fit(X_train_full, train["N02BE"])
best_rf_m01ab.fit(X_train_full, train["M01AB"])

# Predict on hold-out test
yhat_n02be = best_rf_n02be.predict(X_test_full)
yhat_m01ab = best_rf_m01ab.predict(X_test_full)

# Compute metrics
metrics_after = {
    "Target": ["N02BE", "M01AB"],
    "MAE_after": [
        mean_absolute_error(test["N02BE"], yhat_n02be),
        mean_absolute_error(test["M01AB"], yhat_m01ab),
    ],
    "RMSE_after": [
        np.sqrt(mean_squared_error(test["N02BE"], yhat_n02be)),
        np.sqrt(mean_squared_error(test["M01AB"], yhat_m01ab)),
    ],
    "MAPE_after(%)": [
        mape(test["N02BE"].values, yhat_n02be),
        mape(test["M01AB"].values, yhat_m01ab),
    ],
}

# Baseline (Day 2 RF) — sabit rakamlar
baseline = {
    "N02BE": {"MAE": 35.6538, "RMSE": 49.6175, "MAPE(%)": 18.0679},
    "M01AB": {"MAE": 6.4685, "RMSE": 8.5978, "MAPE(%)": 23.1413},
}

df_after = pd.DataFrame(metrics_after)
df_after["MAE_before"] = df_after["Target"].map(lambda t: baseline[t]["MAE"])
df_after["RMSE_before"] = df_after["Target"].map(lambda t: baseline[t]["RMSE"])
df_after["MAPE_before(%)"] = df_after["Target"].map(lambda t: baseline[t]["MAPE(%)"])

# Improvement (% ↓)
df_after["MAE_improve(%)"] = (
    100 * (df_after["MAE_before"] - df_after["MAE_after"]) / df_after["MAE_before"]
)
df_after["RMSE_improve(%)"] = (
    100 * (df_after["RMSE_before"] - df_after["RMSE_after"]) / df_after["RMSE_before"]
)
df_after["MAPE_improve(pp)"] = df_after["MAPE_before(%)"] - df_after["MAPE_after(%)"]

# Düzenli görünüm
cols = [
    "Target",
    "MAE_before",
    "MAE_after",
    "MAE_improve(%)",
    "RMSE_before",
    "RMSE_after",
    "RMSE_improve(%)",
    "MAPE_before(%)",
    "MAPE_after(%)",
    "MAPE_improve(pp)",
]
summary_after = df_after[cols].round(3)
display(summary_after)

# Kaydet
out_csv = RESULTS_DIR / "rf_fullsafe_before_after.csv"
summary_after.to_csv(out_csv, index=False)
print(f"Saved: {out_csv}")

,Target,MAE_before,MAE_after,MAE_improve(%),RMSE_before,RMSE_after,RMSE_improve(%),MAPE_before(%),MAPE_after(%),MAPE_improve(pp)
0,N02BE,35.654,34.607,2.936,49.618,51.505,-3.803,18.068,17.894,0.174
1,M01AB,6.468,6.478,-0.152,8.598,8.543,0.636,23.141,23.226,-0.085


Saved: ..\results\comparison\rf_fullsafe_before_after.csv


## 4) Randomized Search on Reduced-safe Feature Set
Rationale: Use only high-contribution lags (from permutation importance) + minimal calendar to reduce noise and improve generalization, especially for peaks (RMSE).


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from pathlib import Path


def neg_rmse(y_true, y_pred):
    return -np.sqrt(mean_squared_error(y_true, y_pred))


rmse_scorer = make_scorer(neg_rmse, greater_is_better=False)

# Slightly narrower search space to be fast
param_dist_reduced = {
    "n_estimators": [200, 300, 400, 600],
    "max_depth": [None, 8, 12, 16],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", 0.7],
    "bootstrap": [True],
    "random_state": [42],
}


def tune_on_reduced(target, X_train_red, y_train, cv, n_iter=25):
    rf = RandomForestRegressor(n_jobs=-1, random_state=42)
    rs = RandomizedSearchCV(
        rf,
        param_distributions=param_dist_reduced,
        n_iter=n_iter,
        scoring=rmse_scorer,
        cv=cv,
        n_jobs=-1,
        refit=True,
        random_state=42,
        verbose=0,
    )
    rs.fit(X_train_red, y_train)
    print(f"[{target}] Reduced-safe best CV RMSE: {-rs.best_score_:.3f}")
    print(f"[{target}] Best params: {rs.best_params_}")
    return rs


# Tune for both targets on reduced features
rs_red_n02be = tune_on_reduced("N02BE", X_train_reduced, train["N02BE"], tscv)
rs_red_m01ab = tune_on_reduced("M01AB", X_train_reduced, train["M01AB"], tscv)

# Retrain on full train (refit=True zaten var) ve testte değerlendir
best_red_n02be = rs_red_n02be.best_estimator_.fit(X_train_reduced, train["N02BE"])
best_red_m01ab = rs_red_m01ab.best_estimator_.fit(X_train_reduced, train["M01AB"])

yhat_red_n02be = best_red_n02be.predict(X_test_reduced)
yhat_red_m01ab = best_red_m01ab.predict(X_test_reduced)


def mape(y_true, y_pred):
    eps = 1e-8
    denom = np.clip(np.abs(y_true), eps, None)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100


summary_red = pd.DataFrame(
    {
        "Target": ["N02BE", "M01AB"],
        "MAE_fullsafe_after": [
            35.654,
            6.468,
        ],  # will be replaced below with actual 'after' from your table if desired
        "RMSE_fullsafe_after": [
            49.618,
            8.598,
        ],  # placeholder baseline; compare consistently
        "MAPE_fullsafe_after(%)": [
            18.068,
            23.141,
        ],  # placeholder baseline; adjust as needed
    }
)

# Gerçek karşılaştırma: "full-safe AFTER" yerine az önce elde ettiğin AFTER değerlerini kullanmak için:
summary_red = pd.DataFrame(
    {
        "Target": ["N02BE", "M01AB"],
        "MAE_fullsafe_after": [34.607, 6.478],
        "RMSE_fullsafe_after": [51.505, 8.543],
        "MAPE_fullsafe_after(%)": [17.894, 23.226],
    }
)

summary_red["MAE_reduced_after"] = [
    mean_absolute_error(test["N02BE"], yhat_red_n02be),
    mean_absolute_error(test["M01AB"], yhat_red_m01ab),
]
summary_red["RMSE_reduced_after"] = [
    np.sqrt(mean_squared_error(test["N02BE"], yhat_red_n02be)),
    np.sqrt(mean_squared_error(test["M01AB"], yhat_red_m01ab)),
]
summary_red["MAPE_reduced_after(%)"] = [
    mape(test["N02BE"].values, yhat_red_n02be),
    mape(test["M01AB"].values, yhat_red_m01ab),
]

# İyileşme (reduced vs full-safe AFTER)
summary_red["MAE_gain_vs_full_after(%)"] = (
    100
    * (summary_red["MAE_fullsafe_after"] - summary_red["MAE_reduced_after"])
    / summary_red["MAE_fullsafe_after"]
)

summary_red["RMSE_gain_vs_full_after(%)"] = (
    100
    * (summary_red["RMSE_fullsafe_after"] - summary_red["RMSE_reduced_after"])
    / summary_red["RMSE_fullsafe_after"]
)

summary_red["MAPE_gain_vs_full_after(pp)"] = (
    summary_red["MAPE_fullsafe_after(%)"] - summary_red["MAPE_reduced_after(%)"]
)

summary_red = summary_red.round(3)
display(summary_red)

[N02BE] Reduced-safe best CV RMSE: -56.545
[N02BE] Best params: {'random_state': 42, 'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 12, 'bootstrap': True}
[M01AB] Reduced-safe best CV RMSE: -9.981
[M01AB] Best params: {'random_state': 42, 'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.7, 'max_depth': 16, 'bootstrap': True}


,Target,MAE_fullsafe_after,RMSE_fullsafe_after,MAPE_fullsafe_after(%),MAE_reduced_after,RMSE_reduced_after,MAPE_reduced_after(%),MAE_gain_vs_full_after(%),RMSE_gain_vs_full_after(%),MAPE_gain_vs_full_after(pp)
0,N02BE,34.607,51.505,17.894,34.528,50.746,17.865,0.228,1.474,0.029
1,M01AB,6.478,8.543,23.226,6.099,8.197,22.139,5.849,4.048,1.087


In [7]:
def smape(y_true, y_pred):
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2
    eps = 1e-8
    denom = np.clip(denom, eps, None)
    return np.mean(np.abs(y_true - y_pred) / denom) * 100


def wape(y_true, y_pred):
    eps = 1e-8
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true) + eps) * 100


# calculate
for name, y_true, y_pred in [
    ("N02BE", test["N02BE"], yhat_red_n02be),
    ("M01AB", test["M01AB"], yhat_red_m01ab),
]:
    print(
        f"{name}: sMAPE={smape(y_true, y_pred):.2f}%, WAPE={wape(y_true, y_pred):.2f}%"
    )

N02BE: sMAPE=16.18%, WAPE=16.30%
M01AB: sMAPE=17.42%, WAPE=16.64%
